# Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
%load_ext lab_black

In [ ]:
import filteradapt
import pdal
import json

The next step you will typically do is to construct a dataset object by reading a LAS/LAZ file:

The data set can be visualized in the notebook through its `show()` method:

In [ ]:
from filteradapt.paths import locate_file

filename = locate_file("data/500k_NZ20_Westport.laz")


def initilize_pipeline(filename):
    """Initilizes dict fir pipeline generation
    containes list with only filename as entry for "pipeline"


    keyword arguments:
    filename [str]: filename or path to the laz or las file

    """
    if filename[-3:] != ("laz" or "las"):
        raise TypeError("Filetype must be .las or .laz, but is ." + filename[-3:])

    my_json = {"pipeline": [filename]}
    return json.dumps(my_json)


def add_filter_to_pipeline(pipeline, filtername, arg_str=None, arg_option=None):
    """inputs:
    adds one or many filteres to the pipeline
    convert json back to dict and append to the list under "pipeline"

    plan to add option to change filters
    """

    pipeline_dict = json.loads(pipeline)
    # handle exceptions for wrong input

    if arg_str == None and arg_option != None:
        error = "Can't set arg_option to '{}' without setting arg_str.".format(
            arg_option
        )
        raise ValueError(error)
    elif arg_str != None and arg_option == None:
        error = "Can't set arg_str to '{}'  without setting arg_option.".format(arg_str)
        raise ValueError(error)
    # add filter to pipeline
    if arg_str == None:
        pipeline_dict["pipeline"].append({"type": "filters." + filtername})
    else:
        pipeline_dict["pipeline"].append(
            {"type": "filters." + filtername, arg_str: arg_option}
        )

    return json.dumps(pipeline_dict)


test = initilize_pipeline(filename)
print((test))
test = add_filter_to_pipeline(test, "csf")
# test = add_filter_to_pipeline(
#    test, "range", arg_str="limits", arg_option="Classification[2:2]"
# )

print((test))

In [ ]:
pipeline = pdal.Pipeline(test)
pipeline.validate()
%time n_points = pipeline.execute()
print("Pipeline selected {} points ".format(n_points))

In [ ]:
import pandas as pd

# Load Pipeline output in python objects
arr = pipeline2.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df["X_0"] = df["X"]
df["Y_0"] = df["Y"]
df["Z_0"] = df["Z"]
df["X"] = df["X"] - df["X_0"].min()
df["Y"] = df["Y"] - df["Y_0"].min()
df["Z"] = df["Z"] - df["Z_0"].min()

In [ ]:
import ipyvolume.pylab as p3

fig = p3.figure(width=1000)
fig.xlabel = "Y"
fig.ylabel = "Z"
fig.zlabel = "X"
all_points = p3.scatter(df["Y"], df["Z"], df["X"], color="red", size=0.05)
p3.squarelim()
p3.show()

In [ ]:
print(len(df))
df.head()

In [ ]:
import os 
dir_path = os.path.dirname(os.path.realpath("demo.ipynb"))
print(dir_path)